In [2]:
import os
import sys
from pathlib import Path

import numpy as np
import xgboost as xgb
from rich.console import Console
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error

ROOT_DIR = os.path.dirname(
os.path.dirname(os.path.abspath('')))


sys.path.append(os.path.join(ROOT_DIR, "code"))
from tools.data_loader import TestSet, TestSplit, data_loader
from tools.save import save_as_baseline, save_datasets, save_models
from tools.train import evaluate_models, print_test_samples, train_models, print_problematic_samples
from tools.utils import StructureEncoding, Target, check_xgboost_gpu

# Define global variables
DATA_DIR = os.path.join(ROOT_DIR, "data/")

DATA_PATH = os.path.join(DATA_DIR, "data.csv")


In [3]:
console = Console(record=True)
prompt_user = False

encoding = StructureEncoding.ATOMIC

console.log(f"[bold green]Started pipeline for {encoding}")
target = Target.DELTA_E
test_sets_cfg = [
    TestSet("Parameter gen.", size=0.1, split=TestSplit.ROW),
    TestSet("Structure gen.", size=0.1, split=TestSplit.STRUCTURE),
]

# Data Loading
X_train, y_train, test_sets = data_loader(
    target=target,
    encoding=encoding,
    data_path=DATA_PATH,
    test_sets_cfg=test_sets_cfg,
    console=console,
    remove_ref_rows=True,
)

⠇ Loading data

╭───────── Loaded Dataset ─────────╮
│ Train set: 80%                   │
│ Test set 0 (Parameter gen.): 10% │
│ Test set 1 (Structure gen.): 10% │
│ Total datapoints: 539583         │
│ Size: ~388.5MB                   │
╰──────────────────────────────────╯

In [31]:
base = 10
def magnitude(x):
    return int(np.ceil(np.log(x) / np.log(base)))

def magnitude_inv(x):
    return base**float(x-1)
    # for the regression transformation, taking the prediction minus 1 yields way better MAPE scores
    # I think that's because predicting a result with an order of magnitude superior to the actual value
    # yields an APE error of 10 (1000%) while predicting a result with an order of magnitude inferior to the actual value
    # yields an APE error of at most 1 (100%)

def magnitude_transform(a):
    return np.vectorize(magnitude)(a)

def magnitude_inv_transform(a):
    return np.vectorize(magnitude_inv)(a)

In [23]:
x = 1E-6
magnitude(x)

-5

In [24]:
magnitude_y_train = magnitude_transform(y_train)

In [25]:
print("max magnitude: ", np.max(magnitude_y_train))
print("min magnitude: ", np.min(magnitude_y_train))

max magnitude:  1
min magnitude:  -8


In [26]:
model = RandomForestClassifier()
model.fit(X_train, magnitude_y_train)

RandomForestClassifier()

In [32]:
for test_name, X_test, y_test in test_sets:
    magnitude_y_pred = model.predict(X_test)
    magnitude_y_test = magnitude_transform(y_test)
    score = accuracy_score(magnitude_y_test, magnitude_y_pred)
    mape = mean_absolute_percentage_error(y_test, magnitude_inv_transform(magnitude_y_pred))
    console.print(f"Accuracy {test_name}: {score}")
    console.print(f"MAPE {test_name}: {mape}")

Accuracy Parameter gen.: 0.9729048519218652

MAPE Parameter gen.: 0.7185956245378649

Accuracy Structure gen.: 0.6881515545702717

MAPE Structure gen.: 5.207941311082237

In [33]:
class RegModel:
    def __init__(self, class_model):
        self.class_model = class_model

    def predict(self, X):
        return magnitude_inv_transform(self.class_model.predict(X))
reg_models = {"RandomForest": RegModel(model)}

In [34]:
evaluate_models(reg_models, X_train, y_train, test_sets, console)

⠦ Evaluating RandomForest...

                                RandomForest                                
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃  Loss name  ┃   Train    ┃ Test - Parameter gen. ┃ Test - Structure gen. ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│     MSE     │ 1.5919E-02 │      1.4919E-02       │      1.8785E-02       │
│     MAE     │ 3.0425E-02 │      2.9755E-02       │      3.5115E-02       │
│    MAPE     │ 6.1643E-01 │      7.1860E-01       │      5.2079E+00       │
│ Custom MAPE │ 1.5482E+01 │      7.4086E-01       │      5.7084E+00       │
│   50%-APE   │ 6.9475E-01 │      6.9505E-01       │      7.5448E-01       │
│   90%-APE   │ 8.7405E-01 │      8.7622E-01       │      1.0661E+00       │
└─────────────┴────────────┴───────────────────────┴───────────────────────┘

In [35]:
print_test_samples(reg_models, test_sets, console)

  Test samples - Parameter  
            gen.            
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃   Real    ┃ RandomForest ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ 6.400E-01 │  1.000E-01   │
│ 4.685E-02 │  1.000E-02   │
│ 5.416E-03 │  1.000E-03   │
│ 4.764E-03 │  1.000E-03   │
│ 1.943E-02 │  1.000E-02   │
│ 3.198E-05 │  1.000E-05   │
│ 8.182E-03 │  1.000E-03   │
│ 1.351E-05 │  1.000E-05   │
│ 6.042E-04 │  1.000E-04   │
│ 2.127E-02 │  1.000E-02   │
└───────────┴──────────────┘

  Test samples - Structure  
            gen.            
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃   Real    ┃ RandomForest ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ 3.156E-04 │  1.000E-04   │
│ 1.827E-03 │  1.000E-04   │
│ 7.686E-04 │  1.000E-03   │
│ 6.767E-05 │  1.000E-04   │
│ 1.894E-01 │  1.000E-01   │
│ 2.282E-02 │  1.000E-02   │
│ 2.518E-01 │  1.000E-01   │
│ 1.376E-05 │  1.000E-03   │
│ 4.020E-04 │  1.000E-05   │
│ 4.785E-05 │  1.000E-05   │
└───────────┴──────────────┘